In [1]:
from fastai.text import *
import html

/home/bahram/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/bahram/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/bahram/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/bahram/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/home/bahram/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: nump

In [25]:
import json
import html

In [26]:
PATH = Path('data/')

In [27]:
list(PATH.iterdir())

[PosixPath('data/tmp'),
 PosixPath('data/data'),
 PosixPath('data/models'),
 PosixPath('data/multinli_1.0_train.jsonl'),
 PosixPath('data/multinli_1.0_dev_matched.jsonl'),
 PosixPath('data/multinli_1.0_dev_mismatched.jsonl')]

In [28]:
import jsonlines

In [29]:
files = []
with jsonlines.open('data/multinli_1.0_train.jsonl') as reader:
    for obj in reader:
        files.append(obj)
        

In [30]:
len(files)

392702

In [31]:
files[0]

{'annotator_labels': ['neutral'],
 'genre': 'government',
 'gold_label': 'neutral',
 'pairID': '31193n',
 'promptID': '31193',
 'sentence1': 'Conceptually cream skimming has two basic dimensions - product and geography.',
 'sentence1_binary_parse': '( ( Conceptually ( cream skimming ) ) ( ( has ( ( ( two ( basic dimensions ) ) - ) ( ( product and ) geography ) ) ) . ) )',
 'sentence1_parse': '(ROOT (S (NP (JJ Conceptually) (NN cream) (NN skimming)) (VP (VBZ has) (NP (NP (CD two) (JJ basic) (NNS dimensions)) (: -) (NP (NN product) (CC and) (NN geography)))) (. .)))',
 'sentence2': 'Product and geography are what make cream skimming work. ',
 'sentence2_binary_parse': '( ( ( Product and ) geography ) ( ( are ( what ( make ( cream ( skimming work ) ) ) ) ) . ) )',
 'sentence2_parse': '(ROOT (S (NP (NN Product) (CC and) (NN geography)) (VP (VBP are) (SBAR (WHNP (WP what)) (S (VP (VBP make) (NP (NP (NN cream)) (VP (VBG skimming) (NP (NN work)))))))) (. .)))'}

In [ ]:
# trn_j = json.load((PATH/'multinli_1.0_train.jsonl').open())
# trn_j.keys()

In [ ]:
raw_text = []
fs = list()
for file in ['data/multinli_1.0_train.jsonl', 'data/multinli_1.0_dev_matched.jsonl', 'data/multinli_1.0_dev_mismatched.jsonl']:
    with jsonlines.open(file) as reader:
        for obj in reader:
            fs.append(obj)
    
    #     with open(file) as fp:
#         content = json.load(fp)
#         for item in content:
#             raw_text.append(item[0])
#             raw_text.append(item[1])
# print(len(raw_text))

In [ ]:
for item in fs :
    raw_text.append(item['sentence1'])
    raw_text.append(item['sentence2'])

In [ ]:
len(raw_text)

In [32]:
cols = ['sentence1' , 'sentence2' , 'label' , 'genre']

In [33]:
sen1 = list()
sen2 = list()
label = list()
genre = list()
for f in files :
    sen1.append(f['sentence1'])
    sen2.append(f['sentence2'])
    label.append(f['gold_label'])
    genre.append(f['genre'])

In [36]:
df = pd.DataFrame({'sentence1':sen1, 'sentence2':sen2 , 'label':label , 'genre':genre}, columns=cols)

In [42]:
df.describe()

,sentence1,sentence2,label,genre
count,392702,392702,392702,392702
unique,128144,391501,3,5
top,I blinked.,n/a,contradiction,telephone
freq,12,40,130903,83348


In [ ]:
lm_train, lm_valid = sklearn.model_selection.train_test_split(raw_text, test_size=0.1)
df_trn = pd.DataFrame(lm_train)
df_val = pd.DataFrame(lm_valid)

In [ ]:
BOS = 'x_bos'  # beginning-of-sentence tag

re1 = re.compile(r'  +')

def fixup(x):
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>','u_n').replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))

def get_texts(df):
    texts = df[0].astype(str)
    texts = list(texts.apply(fixup).values)
    texts = f'{BOS} ' + df[0].astype(str)
    tok = Tokenizer().proc_all_mp(partition_by_cores(texts))
    return tok

In [ ]:
tok_trn = np.concatenate(get_texts(df_trn))
tok_val = np.concatenate(get_texts(df_val))

In [ ]:
tok_trn[:30]

In [ ]:
np.save('tok_trn.npy', tok_trn)
np.save('tok_val.npy', tok_val)

In [ ]:
tok_trn = np.load('tok_trn.npy')
tok_val = np.load('tok_val.npy')

In [ ]:
freq = Counter(np.concatenate([tok_trn, tok_val]))
freq.most_common(25)

In [ ]:
len(freq)

In [5]:
from collections import defaultdict

In [ ]:
max_vocab = 70000
min_freq = 1
itos = [o for o, c in freq.most_common(max_vocab) if c>=min_freq]
itos.insert(0, '_pad_')
itos.insert(0, '_unk_')
stoi = defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})

In [ ]:
len(stoi)


In [ ]:
trn_lm = np.array([stoi[p] for p in tok_trn])
val_lm = np.array([stoi[p] for p in tok_val])

In [ ]:
pickle.dump(itos, open('itos.pkl', 'wb'))
np.save('trn_lm.npy', trn_lm)
np.save('val_lm.npy', val_lm)

In [6]:
itos = pickle.load(open('itos.pkl', 'rb'))
trn_lm = np.load('trn_lm.npy')
val_lm = np.load('val_lm.npy')

stoi = defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})
vocab_size = len(itos)
vocab_size

70002

In [7]:
for word in val_lm[:100]:
    print(itos[word], end=" ")

x_bos giving a vaccine at $ 30 a dose would take too much money from other areas . x_bos since the 1940s , preferential tax treatment has been granted to individual retirement accounts x_bos owning and sharing the building and not paying rent times five will save the non - profit agencies about $ 375,000 each year . x_bos the film , directed by steven soderbergh , would be worth seeing just for stamp 's performance , at once rock - hard and goofily blinkered , and for peter fonda 's wittily self - parodic turn as the suspected killer 

In [8]:
em_sz,nh,nl = 400,1150,3
PRE_PATH = PATH/'models'/'wt103'
PRE_LM_PATH = PRE_PATH/'fwd_wt103.h5'
wgts = torch.load(PRE_LM_PATH, map_location=lambda storage, loc: storage)

In [9]:
enc_wgts = to_np(wgts['0.encoder.weight'])
row_m = enc_wgts.mean(0)

itos2 = pickle.load(Path(PRE_PATH/'itos_wt103.pkl').open('rb'))
stoi2 = defaultdict(lambda:-1, {v:k for k,v in enumerate(itos2)})

#fill in the missing values from the old vocab
new_w = np.zeros((vocab_size, em_sz), dtype=np.float32)
for i,w in enumerate(itos):
    r = stoi2[w]
    new_w[i] = enc_wgts[r] if r>=0 else row_m
    
#fix up the wgts with the new values
wgts['0.encoder.weight'] = T(new_w)
wgts['0.encoder_with_dropout.embed.weight'] = T(np.copy(new_w))
wgts['1.decoder.weight'] = T(np.copy(new_w))

In [10]:
wd = 1e-7
bptt = 70
batch_size = 64
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))

trn_dl = LanguageModelLoader(trn_lm, batch_size, bptt)
val_dl = LanguageModelLoader(val_lm, batch_size, bptt)
md = LanguageModelData(PATH, 1, vocab_size, trn_dl, val_dl, bs=batch_size, bptt=bptt)

In [11]:
drops = np.array([0.25, 0.1, 0.2, 0.02, 0.15])*0.7

In [12]:
learner = md.get_model(opt_fn, em_sz, nh, nl, 
    dropouti=drops[0], dropout=drops[1], wdrop=drops[2], dropoute=drops[3], dropouth=drops[4])

learner.metrics = [accuracy]


learner.model.load_state_dict(wgts)

In [ ]:
learner.freeze_to(-1)

In [ ]:
# learner.load('final_15_beta')

In [17]:
lr=2e-3
lrs = lr

In [ ]:
learner.fit(lrs/2, 1, wds=wd, use_clr=(32,2), cycle_len=2)

In [ ]:
learner.save('last_layer')

In [ ]:
learner.unfreeze()

In [ ]:
learner.fit(lrs, 1, wds=wd, use_clr=(20,10), cycle_len=15)

In [ ]:
learner.load('final_15_beta2')

In [ ]:
learner.fit(lrs, 1, wds=wd, use_clr=(20,10), cycle_len=4)

In [ ]:
learner.save('final_15_beta3')

In [18]:
learner.fit(lrs, 1, wds=wd, use_clr=(20,10), cycle_len=10)

epoch      trn_loss   val_loss   accuracy                     
    0      4.10229    3.811491   0.341555  
    1      3.878479   3.628341   0.359682                     
    2      3.767599   3.530051   0.37133                      
    3      3.656632   3.4434     0.382217                     
    4      3.555592   3.370625   0.392491                     
    5      3.536918   3.320708   0.399596                     
    6      3.447704   3.259529   0.409181                     
    7      3.390228   3.210213   0.417305                     
    8      3.301714   3.169954   0.423909                     
    9      3.28352    3.143361   0.428271                     


[array([3.14336]), 0.4282713382538543]

In [19]:
learner.save('final')

In [20]:
torch.save(learner.model, "NLI_LM.pt")


In [21]:
em_sz, nh, nl = 400,1150,3
bptt = 70
max_seq = bptt * 20
cats = 3

NLI_LM = torch.load("NLI_LM.pt")

dps = np.array([0.4,0.5,0.05,0.3,0.4])*0.1
NLI_encoder = MultiBatchRNN(bptt, max_seq, vocab_size, em_sz, nh, nl, stoi["_pad_"], dropouti=dps[0], wdrop=dps[2], dropoute=dps[3], dropouth=dps[4])

NLI_encoder.load_state_dict(NLI_LM[0].state_dict())

In [ ]:
class PoolingLinearClassifier(nn.Module):
    def __init__(self, layers, drops):
        super().__init__()
        self.layers = nn.ModuleList([
            LinearBlock(layers[i], layers[i + 1], drops[i]) for i in range(len(layers) - 1)])

    def pool(self, x, bs, is_max):
        f = F.adaptive_max_pool1d if is_max else F.adaptive_avg_pool1d
        return f(x.permute(1,2,0), (1,)).view(bs,-1)

    def forward(self, input):
        raw_outputs, outputs = input
        output = outputs[-1]
        sl,bs,_ = output.size()
        avgpool = self.pool(output, bs, False)
        mxpool = self.pool(output, bs, True)
        x = torch.cat([output[-1], mxpool, avgpool], 1)
        for l in self.layers:
            l_x = l(x)
            x = F.relu(l_x)
        return l_x, raw_outputs, outputs